# `jaxodi` API examples

In this notebook, we will give some examples of using `jaxodi` API functions.

### 1. `light_curve`

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

from jaxodi.doppler_surface import DopplerWav, DopplerSurface, DopplerSpec
from jaxodi.api import light_curve

from jaxoplanet.experimental.starry.ylm import Ylm

jax.config.update("jax_enable_x64", True)

#### 1. Initiate DopplerWav object

`DopplerWav` class contains attributes and methods related to wavlength grids and interpolations among them. It uses the same default values for some attributes (i.e. default wav).

In [ ]:
# user specified parameter
vsini_max = 60000.0

In [ ]:
wav = DopplerWav(vsini_max=vsini_max)

In [ ]:
# eqx fields
wav

#### 2. Initiate DopplerSpec object

`DopplerSpec` class contains spectral data used in the model internally. 

In [ ]:
# user specified data
# notice the length of the input spectrum should be the the same as of wav.wav0
spec0 = 1.0 - 0.75 * jnp.exp(-0.5 * (wav.wav0 - 643.0) ** 2 / 0.0085 ** 2) 

In [ ]:
spec = DopplerSpec(wav, spec0)

In [ ]:
spec

#### 3. Initiate DopplerSurface object

`DopplerSurface` class contains data related to the star.

In [ ]:
# user specified data
nt = 16
theta = np.append([-180], np.linspace(-90, 90, nt - 1))
veq = 60000.0

In [ ]:
surface = DopplerSurface(theta, inc=np.pi/2, obl=0, veq=veq)

In [ ]:
surface

#### 4. Initiate Ylm object

Import `Ylm` from `jaxoplanet`.

In [ ]:
ydeg = 5
N = (ydeg + 1) ** 2
y_ = np.random.rand(N)
y_[0] = 1.0

In [ ]:
y = Ylm.from_dense(y_)

In [ ]:
y.ell_max

#### 5. Calculate light curve

In [ ]:
lc = light_curve(y, wav, spec, surface)

In [ ]:
lc.shape

#### 6. Comparsion with starry method

In [ ]:
import starry
from jaxoplanet.test_utils import assert_allclose

In [ ]:
# initiate starry doppler map using the same input parameters as jaxodi objects
map_ = starry.DopplerMap(ydeg, veq=veq, vsini_max=vsini_max, inc=90, nt=nt, lazy=False)
map_[:, :] = y_
map_.spectrum = 1.0 - 0.75 * np.exp(-0.5 * (map_.wav0 - 643.0) ** 2 / 0.0085 ** 2) 
theta = np.append([-180], np.linspace(-90, 90, nt - 1))

In [ ]:
lc_starry = map_.flux(theta, normalize=False)

In [ ]:
assert_allclose(lc, lc_starry)